# Portal Scrape for iboss

In [ ]:
from getpass import getpass
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
#import plotly.figure_factory as ff

In [ ]:
email = 'xxx' #change to your email
pw = getpass("Enter password: ")
beginlink = "portal link"
loginurl = beginlink + "phantomportal"
protected = "findOrders link"
invlink = "manageOrder?orderId= link"
payload = {
            'button' : 'Log In',
            'userName': email, 
            'userPassword': pw
        }

findOrders = {
                'button' : 'FindClear',
                'orderStatus' : '2',
                'cloudItems' : '-1',
                'maxItemsToReturn' : '500'
        }

headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}

# Click here and press 'Run All Below' to refresh

In [ ]:
with requests.Session() as s:
    p = s.get(loginurl, headers=headers)
    p = s.post(loginurl, data=payload)

    r = s.get(protected)
    r = s.post(protected, data=findOrders)

In [ ]:
soup = bs(r.content, "html.parser")
linklist = []

for link in soup.findAll('tr'
                        ):
    linkstr = link.get('ondblclick')
    if type(linkstr) == str:
        linkstr = linkstr.replace('location.href=', '')
        linkstr = linkstr.replace(';', '')
        linkstr = linkstr.replace("'", '')
        linklist.append(linkstr)

In [ ]:
oidpt_list = []
itempt_list = []
notept_list = []
invpt_list = []
datept_list = []
holdpt_list = []
companypt_list = []
countrypt_list = []
methpt_list = []
serialpt_list = []
addrpt_list = []
order_number_temp = []
html_list = []

with requests.Session() as s:
    p = s.get(loginurl, headers=headers)
    p = s.post(loginurl, data=payload)
    
    for linkp in linklist:
        complink = beginlink + linkp
        c = s.get(complink)
        lsoup = bs(c.text, "html.parser")
        souptd = lsoup.find_all('table', width="100%")
        
        for t_part in souptd:
            
            oidpt = t_part.find('a', class_="blue_n big")
            itempt = t_part.find('td', width="233")
            notetr = t_part.find_all('tr')
            findalltr = t_part.find_all('tr')
            serialpt = t_part.find('a', target="_blank")
            invpt = t_part.find('a', href="javascript:copyToClipboard();")
            
            if oidpt is not None:
                oidpt_strip = oidpt.get_text("|", strip=True) #ID: xxxxxxxxxx
                if oidpt_strip not in oidpt_list:
                    
                    oidpt_list.append(oidpt_strip)
                    
                    itempt_strip = itempt.get_text("|", strip=True) #Local Gateway Node 14700
                    itempt_list.append(itempt_strip)
                    
                    for fatr_part in findalltr:
                        fatrp_strip = fatr_part.get_text("|", strip=True)
                        dateStatus = holdStatus = comStatus = isoStatus = addrStatus = methStatus = snStatus = "?"
                        dateCont = holdCont = comCont = isoCont = addrCont = methCont = snCont = True
                        addradd = False
                        addrstr = ""
                        for dateinfo in fatrp_strip.split("|"):
                            if dateCont is True:
                                if dateStatus == "Status Date:":
                                    datev = dateinfo
                                    dateCont = False
                                else:
                                    dateStatus = dateinfo
                        for holdinfo in fatrp_strip.split("|"):
                            if holdCont is True:
                                if holdStatus == "Order On Hold:":
                                    holdv = holdinfo
                                    holdCont = False
                                else:
                                    holdStatus = holdinfo
                        for cominfo in fatrp_strip.split("|"):
                            if comCont is True:
                                if comStatus == "Shipping Company Name:":
                                    comv = cominfo
                                    comCont = False
                                else:
                                    comStatus = cominfo
                        for isoinfo in fatrp_strip.split("|"):
                            if isoCont is True:
                                if isoStatus == "Shipping Country:":
                                    if len(isoinfo) < 3:
                                        isov = isoinfo
                                        isoCont = False
                                    else:
                                        isov = "EMPTY"
                                        isoCont = False
                                else:
                                    isoStatus = isoinfo
                        for methinfo in fatrp_strip.split("|"):
                            if methCont is True:
                                if methStatus == "Requested Shipping Method:":
                                    methv = methinfo
                                    methCont = False
                                else:
                                    methStatus = methinfo
                        for addrinfo in fatrp_strip.split("|"):
                            if addrCont is True:
                                if addrStatus == "Shipping Address:":
                                    addrStatus = addrinfo
                                    addrstr += (addrStatus + ", ")
                                    addradd = True
                                elif addradd is True:
                                    addrStatus = addrinfo
                                    if addrStatus == "Shipping Country:":
                                        addrstr = addrstr[:-2]
                                        addrv = addrstr
                                        addrCont = False
                                    elif "Shipping" and ":" not in addrinfo:
                                        addrstr += (addrStatus + ", ")
                                else:
                                    addrStatus = addrinfo
                                    
                    notestr = ""
                    isTrue = False
                    for notetr_part in notetr:
                        isRepeated = False
                        notetrp_strip = notetr_part.get_text("|", strip=True) #SHIP TO:
                                                                            #Company Address Here
                                        
                        if oidpt_strip not in order_number_temp:
                            invptEmpty = True
                            for n_strip_p in notetrp_strip.split("|"):
                                if "Note:" in n_strip_p:
                                    notestr += (n_strip_p + " ")
                                    isTrue = True
                                    order_number_temp.append(oidpt_strip)
                                elif "Shipments" == n_strip_p:
                                    noteIsEmpty = notestr
                                    notept_list.append(notestr)
                                    isTrue = False
                                elif isTrue is True:
                                    notestr += (n_strip_p + " ")
                                    
                                if invpt is None:
                                    if invptEmpty is True:
                                        notept_list.append(noteIsEmpty)
                                        invptEmpty = False
                                        order_number_temp.append(oidpt_strip)
                                    
                            if invpt is None:
                                datept_list.append(datev)
                                holdpt_list.append(holdv)
                                companypt_list.append(comv)
                                countrypt_list.append(isov)
                                methpt_list.append(methv)
                                addrpt_list.append(addrv)
                            else:
                                datept_list.append(datev)
                                holdpt_list.append(holdv)
                                companypt_list.append(comv)
                                countrypt_list.append(isov)
                                methpt_list.append(methv)
                                addrpt_list.append(addrv)
                            
                    if serialpt is None:
                        serialpt_list.append("CLOUD NODE")
                    elif serialpt is not None:
                        snv = serialpt.get_text("|", strip=True)
                        if len(snv) >= 1:
                            serialpt_list.append(snv)
                        else:
                            serialpt_list.append("EMPTY NODE")
                            
                    if invpt is None:
                        invpt_list.append(invpt_strip)
                        html_list.append(invlink+invpt_strip[4:])
                    else:
                        invpt_strip = invpt.get_text("|", strip=True) #INV-NUMBER
                        invpt_list.append(invpt_strip)
                        html_list.append(invlink+invpt_strip[4:])

In [ ]:
data = {
    'Date': datept_list,
    'INV': invpt_list,
    'Hold': holdpt_list,
    'Country': countrypt_list,
    'Company': companypt_list,
    'Address': addrpt_list,
    'Method': methpt_list,
    'Item': itempt_list,
    'Note': notept_list,
    'Serial': serialpt_list,
    'Order': oidpt_list,
    'html': html_list
}

pd.set_option('display.max_rows', None)
#pd.set_option("display.max_rows", None, "display.max_columns", None)
df = pd.DataFrame(data)

def convert(dete):
    return '<a href="{}">{}</a>'.format(dete['html'],  dete['Order'])
df['Order'] = df.apply(convert, axis=1)
del df["html"]

In [ ]:
pd.set_option('max_colwidth', 1000)

left_df = df.style.set_properties(**{'text-align': 'left'})
display(left_df)
#fig = ff.create_table(df)
#fig.show()

In [ ]:
raise SystemExit("END")

# Sort table here

In [ ]:
col_name = input("Enter column name: ")
asc_tf = input("Ascending, T/F?: ")

if asc_tf == 'T':
    asc_value = True
else:
    asc_value = False
sortdf = df.sort_values(by=col_name, ascending=asc_value)
sortdf.style.set_properties(**{'text-align': 'left'})

# Filter table here

In [ ]:
column_input = input("Enter column name: ")
value_input = input("Enter value: ")

filter_df = df[(str(value_input) == df[str(column_input)])]

left_fdf = filter_df.style.set_properties(**{'text-align': 'left'})
display(left_fdf)

# Download portal table as csv file

In [ ]:
randnum = random.randrange(1,999999)
df.to_csv (r'C:/Users/'+email[:-10]+'/Downloads/'+str(randnum)+'.csv', index = False, header=True)